In [1]:
# write svm for classification using joint_angles_per_image.csv file
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Load the CSV file
#df = pd.read_csv("joint_angles_and_distances_per_image.csv")
df = pd.read_csv("hand_crafted_features_train.csv")
# Display the first few rows of the dataframe
df.head()




,is_standing,is_elbow_straight,is_elbow_ninety,is_shoulder_air,is_shoulder_adjoining,Left Shoulder_position_x,Left Shoulder_position_y,Right Shoulder_position_x,Right Shoulder_position_y,Left Elbow_position_x,...,Right Hip-Right Knee_distance,Right Hip-Left Ankle_distance,Right Hip-Right Ankle_distance,Left Knee-Right Knee_distance,Left Knee-Left Ankle_distance,Left Knee-Right Ankle_distance,Right Knee-Left Ankle_distance,Right Knee-Right Ankle_distance,Left Ankle-Right Ankle_distance,Label
0,0.0,0.0,0.0,0.0,0.0,665.508057,804.950684,527.564636,722.201660,689.924194,...,295.692230,458.102997,366.550507,208.989395,240.535431,60.458260,449.424255,258.109863,200.321472,bench press
1,0.5,0.0,1.0,1.0,0.0,630.867432,956.674438,595.199951,844.819885,828.369507,...,1006.176758,1006.176758,1006.176758,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,bench press
2,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.000000,554.233704,...,228.502716,330.362244,388.544006,86.897583,279.853516,314.732910,283.994385,297.558807,76.299881,bench press
3,0.0,0.0,0.0,0.0,0.0,737.802368,988.001831,511.211731,1103.709229,918.739929,...,261.809296,423.711060,373.981567,413.173767,250.807648,478.134064,300.248840,112.172729,305.913971,bench press
4,0.0,1.0,0.0,0.0,0.0,677.520142,943.057007,680.474487,889.215454,752.390381,...,375.177582,218.661377,516.164185,218.090225,157.775238,393.502533,229.188934,224.437302,304.515991,bench press


In [2]:
df.shape    

(9381, 96)

In [3]:

df.isnull().sum().sort_values(ascending=False)

is_standing                              0
is_elbow_straight                        0
Left Wrist-Left Knee_distance            0
Left Wrist-Right Hip_distance            0
Left Wrist-Left Hip_distance             0
                                        ..
Left Shoulder-Right Shoulder_distance    0
Right Ankle_position_y                   0
Right Ankle_position_x                   0
Left Ankle_position_y                    0
Label                                    0
Length: 96, dtype: int64

In [4]:
# print dropped columns 

dropped_columns = df.columns[df.isnull().sum() > 0]
print(f"Dropped columns: {dropped_columns}")

Dropped columns: Index([], dtype='object')


In [5]:
# drop nan columns
df = df.dropna(axis=1)


In [6]:
df.shape

(9381, 96)

In [7]:
df.isnull().sum().sum()

0

In [8]:
# print df's class distribution
print(df['Label'].value_counts())

Label
push up         1979
lat pulldown    1942
hammer curl     1906
plank           1841
bench press     1713
Name: count, dtype: int64


In [9]:
# make all label's count equal to the minimum count
# randomly select rows from each class to make the count equal to the minimum count
min_count = df['Label'].value_counts().min()
dfs = []
for label in df['Label'].unique():
    dfs.append(df[df['Label'] == label].sample(min_count))
df = pd.concat(dfs)

# print df's class distribution
print(df['Label'].value_counts())
# save the new dataframe to a new csv file
df.to_csv("joint_angles_and_distances_per_image_balanced.csv", index=False)

Label
bench press     1713
hammer curl     1713
lat pulldown    1713
plank           1713
push up         1713
Name: count, dtype: int64


In [10]:
# encode the labels using LabelEncoder
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['Label'] = le.fit_transform(df['Label'])

# save the encoder to use it later
import joblib

joblib.dump(le, "models/label_encoder.joblib")


['models/label_encoder.joblib']

In [11]:
# Split the data into features (X) and labels (y)
X = df.drop(columns=["Label"])  # Features
y = df["Label"]  # Labels

# normalize the data except the label using standard scaler
from sklearn.preprocessing import StandardScaler
standardScaler = StandardScaler()
X = standardScaler.fit_transform(X)
# save the standard scaler for future use
import joblib
joblib.dump(standardScaler, "models/standard_scaler.joblib")

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
# train the model with xg boost
from xgboost import XGBClassifier


model = XGBClassifier()
model.fit(X_train, y_train)

# save the model
joblib.dump(model, "models/xgboost_model.joblib")

# Make predictions
y_pred = model.predict(X_test)

# classification report
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       373
           1       1.00      1.00      1.00       335
           2       0.99      1.00      1.00       333
           3       1.00      0.99      1.00       351
           4       0.99      1.00      1.00       321

    accuracy                           1.00      1713
   macro avg       1.00      1.00      1.00      1713
weighted avg       1.00      1.00      1.00      1713

[[372   0   1   0   0]
 [  0 335   0   0   0]
 [  0   1 332   0   0]
 [  1   0   0 348   2]
 [  0   0   1   0 320]]


In [13]:
# train and test svm model
from sklearn.svm import SVC
model = SVC()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# classification report and confusion matrix
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

# save the model
joblib.dump(model, "models/svm_model.joblib")



              precision    recall  f1-score   support

           0       0.92      0.97      0.95       373
           1       1.00      1.00      1.00       335
           2       0.97      0.94      0.95       333
           3       0.99      0.98      0.98       351
           4       0.99      0.97      0.98       321

    accuracy                           0.97      1713
   macro avg       0.97      0.97      0.97      1713
weighted avg       0.97      0.97      0.97      1713

[[361   0   9   3   0]
 [  0 335   0   0   0]
 [ 20   1 312   0   0]
 [  3   0   0 345   3]
 [  7   0   1   2 311]]


['models/svm_model.joblib']

In [14]:
# train and test random forest model
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# classification report and confusion matrix
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

# save the model
joblib.dump(model, "models/random_forest_model.joblib")


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       373
           1       1.00      1.00      1.00       335
           2       1.00      1.00      1.00       333
           3       0.99      0.99      0.99       351
           4       1.00      0.99      1.00       321

    accuracy                           1.00      1713
   macro avg       1.00      1.00      1.00      1713
weighted avg       1.00      1.00      1.00      1713

[[372   0   0   1   0]
 [  0 335   0   0   0]
 [  0   1 332   0   0]
 [  1   0   0 349   1]
 [  0   0   1   1 319]]


['models/random_forest_model.joblib']

In [15]:
# train mlp model
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(hidden_layer_sizes=(250, 150,75,50,25), max_iter=1000)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# classifitaion report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

# save the model
joblib.dump(model, "models/mlp_model.joblib")

Accuracy: 99.24%
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       373
           1       1.00      1.00      1.00       335
           2       1.00      0.97      0.98       333
           3       1.00      1.00      1.00       351
           4       1.00      1.00      1.00       321

    accuracy                           0.99      1713
   macro avg       0.99      0.99      0.99      1713
weighted avg       0.99      0.99      0.99      1713



['models/mlp_model.joblib']

# validation part

### load data

In [47]:
import pandas as pd
# Load the CSV file
df = pd.read_csv("hand_crafted_features_test.csv")


In [48]:
# make all label's count equal to the minimum count
# randomly select rows from each class to make the count equal to the minimum count
min_count = df['Label'].value_counts().min()
dfs = []
for label in df['Label'].unique():
    dfs.append(df[df['Label'] == label].sample(min_count))
df = pd.concat(dfs)

# print df's class distribution
print(df['Label'].value_counts())
# save the new dataframe to a new csv file
df.to_csv("validation_dataset_balanced.csv", index=False)

Label
push up         735
plank           735
lat pulldown    735
hammer curl     735
bench press     735
Name: count, dtype: int64


In [49]:
df.dropna(axis=1,inplace=True)

In [50]:
df.shape

(3675, 96)

In [51]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import joblib

# encode the labels using LabelEncoder
# load the encoder
label_encoder = joblib.load("models/label_encoder.joblib")
df["Label"] = label_encoder.transform(df["Label"])

# Split the data into features (X) and labels (y)
#X_val = df.drop(columns=["Image", "Label"])  # Features
X_val = df.drop(columns=["Label"])  # Features
y_val = df["Label"]  # Labels

# normalize the data except the label using standard scaler
# load model
standardScaler = joblib.load("models/standard_scaler.joblib")
X_val = standardScaler.fit_transform(X_val)
# save the standard scaler for future use
import joblib
joblib.dump(standardScaler, "models/standard_scaler.joblib")




['models/standard_scaler.joblib']

In [52]:
y_val

63      4
609     4
86      4
113     4
412     4
       ..
3498    0
3136    0
3090    0
3281    0
3213    0
Name: Label, Length: 3675, dtype: int32

In [21]:
X_val.shape

(3675, 95)

### test models

In [22]:
X_val.shape

(3675, 95)

In [23]:
# test the xgboost model
model = joblib.load("models/xgboost_model.joblib")
y_pred = model.predict(X_val)
print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred))


              precision    recall  f1-score   support

           0       0.34      0.24      0.28       735
           1       0.62      0.41      0.50       735
           2       0.32      0.48      0.38       735
           3       0.26      0.28      0.27       735
           4       0.50      0.51      0.51       735

    accuracy                           0.39      3675
   macro avg       0.41      0.39      0.39      3675
weighted avg       0.41      0.39      0.39      3675

[[176  40 411 108   0]
 [ 28 305  32 115 255]
 [ 86  64 354 113 118]
 [154  81 293 207   0]
 [ 72   3  25 260 375]]


In [24]:
# test the svm model
model = joblib.load("models/svm_model.joblib")
y_pred = model.predict(X_val)
print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred))


              precision    recall  f1-score   support

           0       0.58      0.75      0.66       735
           1       0.62      0.75      0.68       735
           2       0.47      0.33      0.39       735
           3       0.37      0.36      0.37       735
           4       0.29      0.24      0.26       735

    accuracy                           0.49      3675
   macro avg       0.47      0.49      0.47      3675
weighted avg       0.47      0.49      0.47      3675

[[553  50  68   5  59]
 [ 16 553   0   0 166]
 [ 93 164 242 185  51]
 [ 92  93 135 266 149]
 [193  35  68 266 173]]


In [25]:
# test the random forest model
model = joblib.load("models/random_forest_model.joblib")
y_pred = model.predict(X_val)
print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred))


              precision    recall  f1-score   support

           0       0.32      0.39      0.35       735
           1       0.68      0.58      0.63       735
           2       0.35      0.60      0.44       735
           3       0.26      0.18      0.21       735
           4       0.78      0.42      0.54       735

    accuracy                           0.43      3675
   macro avg       0.48      0.43      0.44      3675
weighted avg       0.48      0.43      0.44      3675

[[284  38 358  55   0]
 [185 427  49  52  22]
 [  6 114 443 110  62]
 [290   0 311 134   0]
 [110  48  97 174 306]]


In [26]:
# test the mlp model
model = joblib.load("models/mlp_model.joblib")
y_pred = model.predict(X_val)
print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred))



              precision    recall  f1-score   support

           0       0.82      0.74      0.78       735
           1       0.80      0.87      0.84       735
           2       0.67      0.72      0.70       735
           3       0.59      0.68      0.63       735
           4       0.70      0.56      0.62       735

    accuracy                           0.71      3675
   macro avg       0.72      0.71      0.71      3675
weighted avg       0.72      0.71      0.71      3675

[[543  17  71  45  59]
 [  5 640   2   3  85]
 [ 48  52 530  85  20]
 [ 42  71 113 499  10]
 [ 21  17  74 215 408]]


In [27]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
from tensorflow.keras.optimizers import Adam

def train_model(X_train, y_train, X_test, y_test, input_shape, epochs=30, batch_size=32, validation_split=0.2, learning_rate=0.001):
    model = tf.keras.Sequential([
        layers.Dense(256, activation='relu', input_shape=(input_shape,)),  # Input layer
        layers.Dropout(0.2),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(5, activation='softmax') 
    ])

    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    # Train the model
    history = model.fit(
        X_train, y_train,
        validation_split=validation_split,  # Use a portion of training data for validation
        epochs=epochs,            # Number of epochs
        batch_size=batch_size,    # Batch size
        verbose=0               # Show training progress
    )

    # Evaluate the model
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy: {test_acc:.4f}")



    return model, history, test_acc, test_loss


In [28]:
# test the mlp model with validation data
# use grid search to find the best hyperparameters
best_model = None
best_accuracy = 0
best_params = None  

from sklearn.model_selection import ParameterGrid

param_grid = {
    "epochs": [30, 50],
    "batch_size": [32, 64, 128],
    "validation_split": [0.1, 0.2, 0.3],
    "learning_rate": [0.001, 0.005, 0.01]
}

for params in ParameterGrid(param_grid):
    model, history, test_acc, test_loss = train_model(X_train, y_train, X_val, y_val, X_train.shape[1])
    if test_acc > best_accuracy:
        best_accuracy = test_acc
        best_model = model
        best_params = params

        





c:\Users\Samet\anaconda3\envs\aiclub\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Test Accuracy: 0.7156
Test Accuracy: 0.7018
Test Accuracy: 0.7048
Test Accuracy: 0.7048
Test Accuracy: 0.7102


KeyboardInterrupt: 

In [29]:
# save the best model as joblib file
joblib.dump(best_model, "models/best_mlp_model.joblib")

['models/best_mlp_model.joblib']

In [87]:
# try different model architectures

def train_model(X_train, y_train, X_test, y_test, input_shape, epochs=30, batch_size=32, validation_split=0.2, learning_rate=0.001):
    model = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(input_shape,)),
    layers.BatchNormalization(),
    
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.1),  # Reduced dropout
    
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    
    layers.Dense(512, activation='relu'), 
    layers.BatchNormalization(),
    layers.Dropout(0.1),  # Reduced dropout

    
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.1),  # Reduced dropout

    
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
        
    layers.Dense(5, activation='softmax')  # Output layer
])

    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    # Train the model
    history = model.fit(
        X_train, y_train,
        validation_split=validation_split,  # Use a portion of training data for validation
        epochs=epochs,            # Number of epochs
        batch_size=batch_size,    # Batch size
        verbose=0               # Show training progress
    )

    # Evaluate the model
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy: {test_acc:.4f}")



    return model, history, test_acc, test_loss

In [88]:
# test the mlp model with validation data
# use grid search to find the best hyperparameters
best_model = None
best_accuracy = 0
best_params = None

from sklearn.model_selection import ParameterGrid

param_grid = {
    "epochs": [30, 50],
    "batch_size": [32, 64, 128],
    "validation_split": [0.1, 0.2, 0.3],
    "learning_rate": [0.001, 0.005, 0.01]
}

for params in ParameterGrid(param_grid):
    model, history, test_acc, test_loss = train_model(X_train, y_train, X_val, y_val, X_train.shape[1])
    if test_acc > best_accuracy:
        best_accuracy = test_acc
        best_model = model
        best_params = params

        





Test Accuracy: 0.6444
Test Accuracy: 0.7097
Test Accuracy: 0.5600
Test Accuracy: 0.6376
Test Accuracy: 0.4427
Test Accuracy: 0.6356
Test Accuracy: 0.5918
Test Accuracy: 0.6824
Test Accuracy: 0.6922
Test Accuracy: 0.5804
Test Accuracy: 0.5478
Test Accuracy: 0.6846
Test Accuracy: 0.7048
Test Accuracy: 0.6517
Test Accuracy: 0.6555
Test Accuracy: 0.6297
Test Accuracy: 0.5537
Test Accuracy: 0.6762
Test Accuracy: 0.6150
Test Accuracy: 0.6218
Test Accuracy: 0.6408
Test Accuracy: 0.6150
Test Accuracy: 0.6424
Test Accuracy: 0.6501
Test Accuracy: 0.6835
Test Accuracy: 0.6185
Test Accuracy: 0.6811
Test Accuracy: 0.6307
Test Accuracy: 0.6207
Test Accuracy: 0.5984
Test Accuracy: 0.6218
Test Accuracy: 0.7072
Test Accuracy: 0.6675
Test Accuracy: 0.5224
Test Accuracy: 0.5619
Test Accuracy: 0.6201
Test Accuracy: 0.6822
Test Accuracy: 0.6770
Test Accuracy: 0.6185
Test Accuracy: 0.7091
Test Accuracy: 0.7167
Test Accuracy: 0.6327
Test Accuracy: 0.5739
Test Accuracy: 0.7273
Test Accuracy: 0.6871
Test Accur

KeyboardInterrupt: 

In [91]:
best_model.summary()

Model: "sequential_74"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_532 (Dense)               │ (None, 64)             │         6,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_322         │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_533 (Dense)               │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_323         │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_286 (Dropout)           │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_534 (Dense)               │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_324         │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_535 (Dense)               │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_325         │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_287 (Dropout)           │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_536 (Dense)               │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_326         │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_537 (Dense)               │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_327         │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_288 (Dropout)           │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_538 (Dense)               │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_328         │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_539 (Dense)               │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,066,897 (4.07 MB)

 Trainable params: 354,693 (1.35 MB)

 Non-trainable params: 2,816 (11.00 KB)

 Optimizer params: 709,388 (2.71 MB)

# aaaaa

In [31]:
import torch
import math
import csv


def calculate_angle(p1, p2, p3):
    # Create vectors
    v1 = p1 - p2
    v2 = p3 - p2
    
    # Calculate dot product and magnitudes
    dot_product = torch.dot(v1, v2)
    magnitude_v1 = torch.norm(v1)
    magnitude_v2 = torch.norm(v2)
    
    # Prevent division by zero
    if magnitude_v1 == 0 or magnitude_v2 == 0:
        return None  # Undefined angle
    
    # Calculate angle in radians
    angle_rad = torch.acos(dot_product / (magnitude_v1 * magnitude_v2))
    # Convert to degrees
    return math.degrees(angle_rad.item())

# Function to calculate Euclidean distance between two points (p1, p2)
def calculate_distance(p1, p2):
    return torch.norm(p1 - p2).item()


In [32]:
def is_standing(dataset, threshold=130):
    p1 = torch.tensor([dataset['Left Hip_position_x'], dataset['Left Hip_position_y']])
    p2 = torch.tensor([dataset['Left Knee_position_x'], dataset['Left Knee_position_y']])
    p3 = torch.tensor([dataset['Left Ankle_position_x'], dataset['Left Ankle_position_y']])
    angle = calculate_angle(p1, p2, p3)
    if angle is None:
        return None
    return angle > threshold

def is_elbow_straight(dataset, threshold=160):
    p1 = torch.tensor([dataset['Left Shoulder_position_x'], dataset['Left Shoulder_position_y']])
    p2 = torch.tensor([dataset['Left Elbow_position_x'], dataset['Left Elbow_position_y']])
    p3 = torch.tensor([dataset['Left Wrist_position_x'], dataset['Left Wrist_position_y']])
    angle = calculate_angle(p1, p2, p3)
    if angle is None:
        return None
    return angle > threshold

def is_elbow_ninety(dataset, threshold1= 60 , threshold2 = 120):
    p1 = torch.tensor([dataset['Left Shoulder_position_x'], dataset['Left Shoulder_position_y']])
    p2 = torch.tensor([dataset['Left Elbow_position_x'], dataset['Left Elbow_position_y']])
    p3 = torch.tensor([dataset['Left Wrist_position_x'], dataset['Left Wrist_position_y']])
    angle = calculate_angle(p1, p2, p3)
    if angle is None:
        return None
    return angle > threshold1 and angle < threshold2


def is_shoulder_air(dataset, threshold=160):
    p1 = torch.tensor([dataset['Left Hip_position_x'], dataset['Left Hip_position_y']])
    p2 = torch.tensor([dataset['Left Shoulder_position_x'], dataset['Left Shoulder_position_y']])
    p3 = torch.tensor([dataset['Left Elbow_position_x'], dataset['Left Elbow_position_y']])
    angle = calculate_angle(p1, p2, p3)
    if angle is None:
        return None
    return angle > threshold

def is_shoulder_adjoining(dataset, threshold=30):
    p1 = torch.tensor([dataset['Left Hip_position_x'], dataset['Left Hip_position_y']])
    p2 = torch.tensor([dataset['Left Shoulder_position_x'], dataset['Left Shoulder_position_y']])
    p3 = torch.tensor([dataset['Left Elbow_position_x'], dataset['Left Elbow_position_y']])
    angle = calculate_angle(p1, p2, p3)
    if angle is None:
        return None
    return angle < threshold


In [97]:
from ultralytics import YOLO
import torch
import math
import csv
import pandas as pd
import joblib
import pandas as pd
import os 
import numpy as np

def preprocess_image(image_path):

    model = YOLO("yolo11x-pose.pt")

    model.to('cuda')        
    results = model.predict(image_path, imgsz=320, conf=0.5)

    # List to store calculated angles and distances for each image
    angles_per_image = []

    # Define joint triplets for angle calculations
    joint_triplets = [
        (5, 7, 9),  # Left Elbow
        (6, 8, 10), # Right Elbow
        (5, 11, 13),# Left Hip
        (6, 12, 14),# Right Hip
        (7, 5, 11), # Left Shoulder
        (8, 6, 12), # Right Shoulder
        (11, 13, 15),# Left Knee
        (12, 14, 16) # Right Knee
    ]

    # Joint labels for reference
    joint_labels = ["Nose", "Left Eye", "Right Eye", "Left Ear", "Right Ear",
                "Left Shoulder", "Right Shoulder", "Left Elbow", "Right Elbow",
                "Left Wrist", "Right Wrist", "Left Hip", "Right Hip",
                "Left Knee", "Right Knee", "Left Ankle", "Right Ankle"]
    # Iterate over each image in the batch

    for img_idx in range(len(results)):  # Iterate over images (batch size)
        if len(results[img_idx]) == 0:
            print(f"No keypoints detected in image {img_idx + 1}")
            continue
        if len(results[img_idx]) > 1:
            print(f"Multiple detections in image {img_idx + 1}")
            continue
        image_angles = {"Image": img_idx + 1}
        
        # Calculate angles and distances for each joint triplet
        for triplet in joint_triplets:
            p1 = results[img_idx].keypoints.xy[0][triplet[0]]  # Correctly access the joint coordinates
            p2 = results[img_idx].keypoints.xy[0][triplet[1]]
            p3 = results[img_idx].keypoints.xy[0][triplet[2]]
            
            # Make sure the coordinates are tensors and have the shape [2] (x, y)
            angle = calculate_angle(p1, p2, p3)
            image_angles[f"{joint_labels[triplet[0]]}-{joint_labels[triplet[1]]}-{joint_labels[triplet[2]]}"] = angle
        
        # Calculate and add distances for each joint pair
        for i in range(len(results[img_idx].keypoints.xy[0])):
            for j in range(i + 1, len(results[img_idx].keypoints.xy[0])):
                p1 = results[img_idx].keypoints.xy[0][i]
                p2 = results[img_idx].keypoints.xy[0][j]
                distance = calculate_distance(p1, p2)
                image_angles[f"{joint_labels[i]}-{joint_labels[j]}_distance"] = distance

        #add the positions of the joints
        for i in range(len(results[img_idx].keypoints.xy[0])):
            p1 = results[img_idx].keypoints.xy[0][i]
            y_pos = p1[1]
            # convert tensor to float
            y_pos = y_pos.item()

            x_pos = p1[0]
            # convert tensor to float
            x_pos = x_pos.item()

            image_angles[f"{joint_labels[i]}_position_y"] = y_pos
            image_angles[f"{joint_labels[i]}_position_x"] = x_pos
            
        
        angles_per_image.append(image_angles)

    # return angles_per_image as a dataframe
    df = pd.DataFrame(angles_per_image)
    dropped_columns = ['Left Shoulder-Left Elbow-Left Wrist',
    'Right Shoulder-Right Elbow-Right Wrist',
    'Left Shoulder-Left Hip-Left Knee',
    'Right Shoulder-Right Hip-Right Knee',
    'Left Elbow-Left Shoulder-Left Hip',
    'Right Elbow-Right Shoulder-Right Hip', 'Left Hip-Left Knee-Left Ankle',
    'Right Hip-Right Knee-Right Ankle', 'Image']
    df = df.drop(columns = dropped_columns)


    hand_crafted_features = pd.DataFrame()

    hand_crafted_features["is_standing"] = df.apply(is_standing, axis=1)
    
    hand_crafted_features["is_elbow_straight"] = df.apply(is_elbow_straight, axis=1)

    hand_crafted_features["is_elbow_ninety"] = df.apply(is_elbow_ninety, axis=1)

    hand_crafted_features["is_shoulder_air"] = df.apply(is_shoulder_air, axis=1)

    hand_crafted_features["is_shoulder_adjoining"] = df.apply(is_shoulder_adjoining, axis=1)


    for column in hand_crafted_features.columns:
        hand_crafted_features[column] = hand_crafted_features[column].apply(lambda x: 0.5 if x is None else 0 if x == False else 1)

    added_joints = ["Left Shoulder", "Right Shoulder", "Left Elbow", "Right Elbow","Left Wrist", "Right Wrist", "Left Hip", "Right Hip",
            "Left Knee", "Right Knee", "Left Ankle", "Right Ankle"]

    # put these positions and distances between them to a hand_crafted_features dataframe
    for joint in added_joints:
        hand_crafted_features[f"{joint}_position_x"] = df[f"{joint}_position_x"]
        hand_crafted_features[f"{joint}_position_y"] = df[f"{joint}_position_y"]

    # put distances between each joint to the dataframe
    for i in range(len(added_joints)):
        for j in range(i + 1, len(added_joints)):
            hand_crafted_features[f"{added_joints[i]}-{added_joints[j]}_distance"] = df[f"{added_joints[i]}-{added_joints[j]}_distance"]

    # scale data using standard scaler saved in models folder
    standardScaler = joblib.load("models/standard_scaler.joblib")
    hand_crafted_features = standardScaler.transform(hand_crafted_features)
    print(hand_crafted_features)
    return hand_crafted_features
    

In [98]:
def predict(image_folder, model_path, encoder_path):
    # Load the model
    model = joblib.load(model_path)
    encoder = joblib.load(encoder_path)

    # Load the image
    image_angles = preprocess_image(image_folder)
    if image_angles is None:
        return None

    # Make predictions
    predictions = model.predict(image_angles)
    print(predictions)
    if predictions is None:
        return None
    # decode the predictions with using the encoder
    predictions = predictions.argmax(axis=1)
    predictions = encoder.inverse_transform(predictions)
    return predictions


In [99]:
# concat two numpy arrays
def concat(a, b):
    return np.concatenate((a, b), axis=1)



In [118]:
# make prediction on the validation dataset
def test_model(dataset_path,model_path,encoder_path):
    y_pred = None
    y_test = None
    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        
        predictions = predict(folder_path, model_path, encoder_path)
        if predictions is None:
            continue
        # create numpy array that holds real values of the image with folder name
        # y_test = np.array(folder name) * len(predictions)
        y_test = np.concatenate((y_test, [folder] * len(predictions)), axis=0) if y_test is not None else [folder] * len(predictions)
        y_pred = np.concatenate((y_pred, predictions), axis=0) if y_pred is not None else predictions
        print(f"Predicted: {predictions} for image {folder}")

    return y_test, y_pred

            
            

In [119]:
y_test, y_pred = test_model("validation_dataset", "models/best_mlp_model.joblib", "models/label_encoder.joblib")


image 1/818 c:\Users\Samet\Desktop\3. Grade 1. Term\AIN 311 - 313\project\yolo model\validation_dataset\bench press\0.00000258.jpg: 320x192 1 person, 69.7ms
image 2/818 c:\Users\Samet\Desktop\3. Grade 1. Term\AIN 311 - 313\project\yolo model\validation_dataset\bench press\0.00226286.jpg: 320x192 (no detections), 31.7ms
image 3/818 c:\Users\Samet\Desktop\3. Grade 1. Term\AIN 311 - 313\project\yolo model\validation_dataset\bench press\0.00241480.jpg: 320x192 (no detections), 39.3ms
image 4/818 c:\Users\Samet\Desktop\3. Grade 1. Term\AIN 311 - 313\project\yolo model\validation_dataset\bench press\0.00377820.jpg: 320x192 1 person, 28.6ms
image 5/818 c:\Users\Samet\Desktop\3. Grade 1. Term\AIN 311 - 313\project\yolo model\validation_dataset\bench press\0.00454068.jpg: 320x192 1 person, 29.7ms
image 6/818 c:\Users\Samet\Desktop\3. Grade 1. Term\AIN 311 - 313\project\yolo model\validation_dataset\bench press\0.00630258.jpg: 320x192 1 person, 28.6ms
image 7/818 c:\Users\Samet\Desktop\3. Grade

In [120]:
# print classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

 bench press       0.78      0.80      0.79       721
 hammer curl       0.95      0.94      0.95       779
lat pulldown       0.72      0.81      0.76       575
       plank       0.49      0.68      0.57       711
     push up       0.78      0.45      0.57       847

    accuracy                           0.73      3633
   macro avg       0.74      0.74      0.73      3633
weighted avg       0.75      0.73      0.73      3633

